In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("salaries.csv")
df.head()

,company,job,degree,salary_more_then_100k
0,google,sales executive,bachelors,0
1,google,sales executive,masters,0
2,google,business manager,bachelors,1
3,google,business manager,masters,1
4,google,computer programmer,bachelors,0


In [4]:
inputs = df.drop('salary_more_then_100k',axis='columns')

In [5]:
target = df['salary_more_then_100k']

In [6]:
from sklearn.preprocessing import LabelEncoder
le_company = LabelEncoder()
le_job = LabelEncoder()
le_degree = LabelEncoder()

In [7]:
inputs['company_n'] = le_company.fit_transform(inputs['company'])
inputs['job_n'] = le_job.fit_transform(inputs['job'])
inputs['degree_n'] = le_degree.fit_transform(inputs['degree'])

In [8]:
inputs

,company,job,degree,company_n,job_n,degree_n
0,google,sales executive,bachelors,2,2,0
1,google,sales executive,masters,2,2,1
2,google,business manager,bachelors,2,0,0
3,google,business manager,masters,2,0,1
4,google,computer programmer,bachelors,2,1,0
5,google,computer programmer,masters,2,1,1
6,abc pharma,sales executive,masters,0,2,1
7,abc pharma,computer programmer,bachelors,0,1,0
8,abc pharma,business manager,bachelors,0,0,0
9,abc pharma,business manager,masters,0,0,1


In [9]:

inputs_n = inputs.drop(['company','job','degree'],axis='columns')

In [10]:
inputs_n

,company_n,job_n,degree_n
0,2,2,0
1,2,2,1
2,2,0,0
3,2,0,1
4,2,1,0
5,2,1,1
6,0,2,1
7,0,1,0
8,0,0,0
9,0,0,1


In [11]:

from sklearn import tree
model = tree.DecisionTreeClassifier()

In [12]:
model.fit(inputs_n, target)

DecisionTreeClassifier()

In [13]:
model.score(inputs_n,target)

1.0

In [18]:
import numpy as np
import pandas as pd

# Function to calculate entropy
def entropy(y):
    values, counts = np.unique(y, return_counts=True)
    probs = counts / counts.sum()
    return -np.sum(probs * np.log2(probs))

# Function to calculate information gain
def info_gain(y, X_column):
    total_entropy = entropy(y)
    values, counts = np.unique(X_column, return_counts=True)
    weighted_entropy = np.sum([(counts[i] / np.sum(counts)) * entropy(y[X_column == v]) for i, v in enumerate(values)])
    return total_entropy - weighted_entropy

# Function to find the best split
def best_split(X, y):
    best_feature = None
    best_gain = -1
    for feature in X.columns:
        gain = info_gain(y, X[feature])
        if gain > best_gain:
            best_gain = gain
            best_feature = feature
    return best_feature

# Recursive function to build the decision tree
def build_tree(X, y):
    if len(np.unique(y)) == 1:
        return np.unique(y)[0]

    if len(X.columns) == 0:
        return np.bincount(y).argmax()

    best_feature = best_split(X, y)
    if best_feature is None:
        return np.bincount(y).argmax()

    tree = {best_feature: {}}
    for value in np.unique(X[best_feature]):
        X_sub = X[X[best_feature] == value].drop(columns=[best_feature])
        y_sub = y[X[best_feature] == value]
        tree[best_feature][value] = build_tree(X_sub, y_sub)

    return tree

# Function to predict using the decision tree
def predict(tree, sample):
    if not isinstance(tree, dict):
        return tree

    root = next(iter(tree))
    value = sample[root]

    if value in tree[root]:
        return predict(tree[root][value], sample)
    else:
        return 0  # Default class if unseen value

df = pd.read_csv("salaries.csv")

# Prepare data for the decision tree
X = df.drop(columns=["salary_more_then_100k"])
y = df["salary_more_then_100k"].values

# Convert categorical variables to numerical labels
for col in X.columns:
    X[col] = X[col].astype("category").cat.codes

# Build the decision tree
decision_tree = build_tree(X, y)





In [20]:
import numpy as np
import pandas as pd

# Function to calculate Gini Index
def gini_index(y):
    values, counts = np.unique(y, return_counts=True)
    probs = counts / counts.sum()
    return 1 - np.sum(probs ** 2)

# Function to calculate information gain using Gini Index
def gini_gain(y, X_column):
    total_gini = gini_index(y)
    values, counts = np.unique(X_column, return_counts=True)
    weighted_gini = np.sum([(counts[i] / np.sum(counts)) * gini_index(y[X_column == v]) for i, v in enumerate(values)])
    return total_gini - weighted_gini

# Function to find the best split
def best_split(X, y):
    best_feature = None
    best_gain = -1
    for feature in X.columns:
        gain = gini_gain(y, X[feature])
        if gain > best_gain:
            best_gain = gain
            best_feature = feature
    return best_feature

# Recursive function to build the decision tree
def build_tree(X, y):
    if len(np.unique(y)) == 1:
        return np.unique(y)[0]

    if len(X.columns) == 0:
        return np.bincount(y).argmax()

    best_feature = best_split(X, y)
    if best_feature is None:
        return np.bincount(y).argmax()

    tree = {best_feature: {}}
    for value in np.unique(X[best_feature]):
        X_sub = X[X[best_feature] == value].drop(columns=[best_feature])
        y_sub = y[X[best_feature] == value]
        tree[best_feature][value] = build_tree(X_sub, y_sub)

    return tree

# Function to predict using the decision tree
def predict(tree, sample):
    if not isinstance(tree, dict):
        return tree

    root = next(iter(tree))
    value = sample[root]

    if value in tree[root]:
        return predict(tree[root][value], sample)
    else:
        return 0  # Default class if unseen value


df = pd.read_csv("salaries.csv")

# Prepare data for the decision tree
X = df.drop(columns=["salary_more_then_100k"])
y = df["salary_more_then_100k"].values

# Convert categorical variables to numerical labels
for col in X.columns:
    X[col] = X[col].astype("category").cat.codes

# Build the decision tree
decision_tree = build_tree(X, y)